In [2]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [ ]:
api.dataset_download_files('alexattia/the-simpsons-characters-dataset', path='./input', unzip=True)

In [ ]:
import splitfolders

input_dir = './input/simpsons_dataset'
output_dir = './data'

splitfolders.ratio(input_dir, output_dir, ratio=(0.9, 0.1))

In [9]:
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/test'

image_size = (64, 64, 3)
epochs = 50
batch = 100

In [10]:
import os

labels = os.listdir(train_dir)

In [ ]:
import cv2
import matplotlib.pyplot as plt

image_name = train_dir + "/apu_nahasapeemapetilon/pic_0123.jpg"

def plot_img(image_location):
    image = cv2.imread(image_location)
    image = cv2.resize(image, (512, 512))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)); plt.axis('off')
    return

plot_img(image_name)


In [31]:
import tensorflow
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size = image_size[:2],
    batch_size = batch,
    class_mode='categorical',
    color_mode='rgb'
)

val_set = test_datagen.flow_from_directory(
    val_dir,
    target_size = image_size[:2],
    batch_size = batch,
    class_mode = 'categorical',
    color_mode = 'rgb'
)

In [ ]:
for data_batch, labels_batch in training_set:
    print(data_batch.shape)
    break

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D, Flatten

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=image_size, activation="relu"))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation='softmax'))

In [ ]:
model.summary()

In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

checkpoint_filepath = './model.h5.keras'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

callbacks = [
    EarlyStopping(patience = 5, monitor = 'val_accuracy', mode='max'),
    model_checkpoint_callback
]

In [ ]:
model.fit(
    training_set,
    epochs = epochs,
    validation_data = val_set,
    callbacks = callbacks
)